In [77]:
import networkx as nx
import numpy as np
import copy

In [84]:
def random_spin_config(graph):
    """
    graph: networkx graph

    returns: a numpy array of length graph.number_of_nodes() with uniformly random sampled -1s & 1s
    """
    n = graph.number_of_nodes()
    spins = np.random.choice([1,-1], size=n)
    return spins
    

In [85]:
# Example
p = nx.petersen_graph()
spins = random_spin_config(p)
print("Spins:", spins)

Spins: [ 1  1 -1 -1  1 -1 -1  1 -1  1]


Is J usually negative in the function below? Cause otherwise the ising hamiltonian is minimized when all spins point in the same direction, which would be lame.

In [86]:
def total_energy(graph, spins, J=-1):
    """
    spins: numpy array of length graph.number_of_nodes()
    """
    E = 0
    for u, v in graph.edges:
        E += spins[u] * spins[v]
    return -J * E

In [107]:
# Example 
p = nx.petersen_graph()
spins = random_spin_config(p)
E = total_energy(p, spins)
print("Energy:", E)

Energy: 1


In [117]:
def update_one(graph, spins, J=-1, beta=1):
    """
    graph: networkx graph
    spins: numpy array of length graph.number_of_nodes()

    Randomly choses a node and tries to flip its spin. The flip is either accepted or rejected.
    Returns the updated spins array.
    """
    u = np.random.randint(len(spins))
    E_start = 0
    E_end = 0
    for v in graph.adj[u]:
        E_start += spins[u] * spins[v]
        E_end -= spins[u] * spins[v] # flip the spin on u
    delta_E = -J * (E_end - E_start)

    if delta_E < 0:
        spins[u] *= -1
        return spins
    elif delta_E > 0 and np.random.random() <= np.e ** (-delta_E*beta):
        spins[u] *= -1
        return spins
    else:
        return spins


In [118]:
# Example
p = nx.petersen_graph()
spins = random_spin_config(p)
old_spins = copy.copy(spins)
print("Initial spin config:", old_spins)
spins = update_one(p, spins)
print("Spin config after one update:", spins)
print("Difference:", spins - old_spins)

Initial spin config: [-1  1 -1  1  1  1 -1 -1 -1  1]
Spin config after one update: [-1  1  1  1  1  1 -1 -1 -1  1]
Difference: [0 0 2 0 0 0 0 0 0 0]


In [119]:
def update_many(graph, spins, num_iter, J=-1, beta=1):
    spins = random_spin_config(p)
    for i in range(num_iter):
        spins = update_one(p, spins, J, beta)
    return spins

In [123]:
# Example
p = nx.petersen_graph()
spins = random_spin_config(p)
print("Initial energy:", total_energy(p, spins))
spins = update_many(p, spins, 1000000)
print("Energy after 1000000 updates:", total_energy(p, spins))
spins = update_many(p, spins, 1000000)
print("Energy after 2000000 updates:", total_energy(p, spins))
spins = update_many(p, spins, 1000000)
print("Energy after 3000000 updates:", total_energy(p, spins))
spins = update_many(p, spins, 1000000)
print("Energy after 4000000 updates:", total_energy(p, spins))
spins = update_many(p, spins, 1000000)
print("Energy after 5000000 updates:", total_energy(p, spins))

Initial energy: 3
Energy after 1000000 updates: -7
Energy after 2000000 updates: -7
Energy after 3000000 updates: -7
Energy after 4000000 updates: -9
Energy after 5000000 updates: -9


It seems to oscilate between -7 and -9. I dont know if that is a bug or not!?